In [2]:
import pandas as pd
import numpy as np

In [205]:
data = pd.read_csv('car_price_prediction.csv', delimiter=',')

In [206]:
data

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19232,45798355,8467,-,MERCEDES-BENZ,CLK 200,1999,Coupe,Yes,CNG,2.0 Turbo,300000 km,4.0,Manual,Rear,02-Mar,Left wheel,Silver,5
19233,45778856,15681,831,HYUNDAI,Sonata,2011,Sedan,Yes,Petrol,2.4,161600 km,4.0,Tiptronic,Front,04-May,Left wheel,Red,8
19234,45804997,26108,836,HYUNDAI,Tucson,2010,Jeep,Yes,Diesel,2,116365 km,4.0,Automatic,Front,04-May,Left wheel,Grey,4
19235,45793526,5331,1288,CHEVROLET,Captiva,2007,Jeep,Yes,Diesel,2,51258 km,4.0,Automatic,Front,04-May,Left wheel,Black,4


## 1. EDA

### 1.0 Изучение данных 

In [207]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19237 entries, 0 to 19236
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                19237 non-null  int64  
 1   Price             19237 non-null  int64  
 2   Levy              19237 non-null  object 
 3   Manufacturer      19237 non-null  object 
 4   Model             19237 non-null  object 
 5   Prod. year        19237 non-null  int64  
 6   Category          19237 non-null  object 
 7   Leather interior  19237 non-null  object 
 8   Fuel type         19237 non-null  object 
 9   Engine volume     19237 non-null  object 
 10  Mileage           19237 non-null  object 
 11  Cylinders         19237 non-null  float64
 12  Gear box type     19237 non-null  object 
 13  Drive wheels      19237 non-null  object 
 14  Doors             19237 non-null  object 
 15  Wheel             19237 non-null  object 
 16  Color             19237 non-null  object

#### Для начала надо обратить внимание, что у нас имеется абсолютно бесполезный признак ID, который не вносит никакой полезной информации для нас, поэтому его стоит удалить

In [208]:
data.drop(columns=['ID'], inplace=True)

#### Также заметим, что у нас большое число признаков типа object. Однако модели классического ML не умеют работать с такими признаками, поэтому их надо привести к числовому типу(???)

#### 1) Сначала преобразуем столбцы, где основную информацию несут цифры(engine volume, mileage doors), путем удаления лишних слов

In [209]:
def vol(value):
    return float(value[:3])
data['Engine volume'] = data['Engine volume'].apply(vol)

In [210]:
def mil(value):
    return float(value[:-3])
data['Mileage'] = data['Mileage'].apply(mil)

In [211]:
def do(value):
    return int(value[1])
data['Doors'] = data['Doors'].apply(do)

In [212]:
def lev(value):
    if value == '-':
        value = 0
    return int(value)
data['Levy'] = data['Levy'].apply(lev)

In [213]:
data['Price'] += data['Levy']

In [214]:
data.drop(columns=['Levy'], inplace=True)

### 2) Далее поработаем с столбцами, где данные придется кодировать

In [215]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['Leather interior'] = le.fit_transform(data['Leather interior'])

#### Далее мы будем работать с one hot encoder и для удобства мы создадим отдельную таблицу только с категориальными столбцами

In [216]:
#возьмем те марки, у которых есть хотя бы 30 машин
manufact_list = data['Manufacturer'].value_counts()[:29].index
manufact_list

Index(['HYUNDAI', 'TOYOTA', 'MERCEDES-BENZ', 'FORD', 'CHEVROLET', 'BMW',
       'LEXUS', 'HONDA', 'NISSAN', 'VOLKSWAGEN', 'SSANGYONG', 'KIA', 'OPEL',
       'MITSUBISHI', 'SUBARU', 'AUDI', 'MAZDA', 'JEEP', 'DAEWOO', 'DODGE',
       'FIAT', 'SUZUKI', 'PORSCHE', 'LAND ROVER', 'VAZ', 'MINI', 'JAGUAR',
       'RENAULT', 'INFINITI'],
      dtype='object')

In [217]:
# возьмем модели, которые есть хотя бы 50 штук
model_list = data['Model'].value_counts()[:64]
model_list

Prius        1083
Sonata       1079
Camry         938
Elantra       922
E 350         542
             ... 
GLE 350        55
Accent         54
Outlander      54
C-MAX          53
XV             51
Name: Model, Length: 64, dtype: int64

In [218]:
def man(value):
    if value in manufact_list:
        return value
    else:
        return '-'
data['Manufacturer'] = data['Manufacturer'].apply(man)

In [219]:
def mod(value):
    if value in model_list:
        return value
    else:
        return '-'
data['Model'] = data['Model'].apply(mod)

In [220]:
data = data[(data['Model'] != '-') & (data['Manufacturer'] != '-')]

In [221]:
mask = (data.dtypes == object).values
cat_features = data[data.columns[mask]]
real_features = data[data.columns[~mask]]

In [222]:
cat_features_cod = pd.get_dummies(cat_features)

In [223]:
data_go = pd.concat([cat_features_cod, real_features], axis=1)

In [224]:
data_go

,Manufacturer_AUDI,Manufacturer_BMW,Manufacturer_CHEVROLET,Manufacturer_DAEWOO,Manufacturer_FORD,Manufacturer_HONDA,Manufacturer_HYUNDAI,Manufacturer_KIA,Manufacturer_LEXUS,Manufacturer_MERCEDES-BENZ,...,Color_White,Color_Yellow,Price,Prod. year,Leather interior,Engine volume,Mileage,Cylinders,Doors,Airbags
0,0,0,0,0,0,0,0,0,1,0,...,0,0,14727,2010,1,3.5,186005.0,6.0,4,12
2,0,0,0,0,0,1,0,0,0,0,...,0,0,8467,2006,0,1.3,200000.0,4.0,4,2
3,0,0,0,0,1,0,0,0,0,0,...,1,0,4469,2011,1,2.5,168966.0,4.0,4,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,12172,2014,1,1.3,91901.0,4.0,4,4
5,0,0,0,0,0,0,1,0,0,0,...,1,0,40384,2016,1,2.0,160931.0,4.0,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19231,0,0,0,0,0,0,0,0,0,1,...,0,0,6857,2013,1,3.5,107800.0,6.0,4,12
19233,0,0,0,0,0,0,1,0,0,0,...,0,0,16512,2011,1,2.4,161600.0,4.0,4,8
19234,0,0,0,0,0,0,1,0,0,0,...,0,0,26944,2010,1,2.0,116365.0,4.0,4,4
19235,0,0,1,0,0,0,0,0,0,0,...,0,0,6619,2007,1,2.0,51258.0,4.0,4,4


In [225]:
y = data_go['Price']
X = data_go.drop(columns=['Price'])

## 2. Построение моделей

### 2.1 Baseline

#### В качестве baseline возьмем самую обычную линейную регрессию

In [228]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [227]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [229]:
lr = LinearRegression().fit(X_train, y_train)
print(f"MSE train: {mean_squared_error(y_train, lr.predict(X_train))}")
print(f"MSE test: {mean_squared_error(y_test, lr.predict(X_test))}")

MSE train: 128869279.00179298
MSE test: 135773559.77564144


### 2.2 Linear regression

#### Но в этот раз поработает с параметрами

In [236]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

import warnings

warnings.filterwarnings('ignore')

In [233]:
pipe = Pipeline(steps=[
    ('variance', VarianceThreshold(0.01)),
    ('selection', SelectFromModel(Ridge(5.0))),
    ('regressor', Ridge(5.0))
])

pipe.fit(X_train, y_train)

/Users/nikitapodkopaev/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=1.5913e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Pipeline(steps=[('variance', VarianceThreshold(threshold=0.01)),
                ('selection', SelectFromModel(estimator=Ridge(alpha=5.0))),
                ('regressor', Ridge(alpha=5.0))])

In [235]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'variance__threshold': [1e-2, 1e-1, 1, 10],
    'selection__estimator__alpha': [1e-2, 1e-1, 1, 10],
    'regressor__alpha': [1, 3, 5]
}
grid_search = GridSearchCV(pipe, param_grid, cv=5)

grid_search.fit(X_train, y_train)

/Users/nikitapodkopaev/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=4.36714e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/nikitapodkopaev/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=4.1845e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/nikitapodkopaev/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=3.53734e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/nikitapodkopaev/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=7.43334e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('variance',
                                        VarianceThreshold(threshold=0.01)),
                                       ('selection',
                                        SelectFromModel(estimator=Ridge(alpha=5.0))),
                                       ('regressor', Ridge(alpha=5.0))]),
             param_grid={'regressor__alpha': [1, 3, 5],
                         'selection__estimator__alpha': [0.01, 0.1, 1, 10],
                         'variance__threshold': [0.01, 0.1, 1, 10]})

In [237]:
param_best = grid_search.best_estimator_

In [238]:
param_best.named_steps

{'variance': VarianceThreshold(threshold=0.01),
 'selection': SelectFromModel(estimator=Ridge(alpha=1)),
 'regressor': Ridge(alpha=1)}

In [239]:
print(f"MSE train: {mean_squared_error(y_train, param_best.predict(X_train))}")
print(f"MSE test: {mean_squared_error(y_test, param_best.predict(X_test))}")

MSE train: 160085826.33468103
MSE test: 168444440.97807705


MSE train: 128869279.00179298
MSE test: 135773559.77564144